In [2]:
project_id = 'nba-predictions-test'

In [7]:
!gcloud init

^C


In [3]:
!gcloud projects create {project_id} --enable-cloud-apis

ERROR: (gcloud.projects.create) Project creation failed. The project ID you specified is already in use by another project. Please try an alternative ID.


In [5]:
!gcloud config list

[accessibility]
screen_reader = False
[compute]
region = us-central1
zone = us-central1-c
[core]
account = cwilbar04@gmail.com
disable_usage_reporting = True
project = nba-predictions-dev



Your active configuration is: [default]


In [6]:
!gcloud deployment-manager deployments list

Listed 0 items.


In [8]:
!gcloud components update

Beginning update. This process may take several minutes.

ERROR: (gcloud.components.update) You cannot perform this action because you do not have permission to modify the Google Cloud SDK installation directory [C:\Program Files (x86)\Google\Cloud SDK\google-cloud-sdk].

Click the Google Cloud SDK Shell icon and re-run the command in that window, or re-run the command with elevated privileges by right-clicking cmd.exe and selecting "Run as Administrator".
